# **<font color = '#3498eb'>OBJETIVES</font>**

Hi 

# **<font color = '#3498eb'>SECTOR, COMPANY</font>**

# **<font color = '#3498eb'>SOME QUESTIONS TO START THE ANALYSIS</font>**

# **<font color = '#3498eb'>LIBRARIES</font>**

In [ ]:
%pip install tabulate
%pip install mysql-connector-python
%pip install pymysql

In [5]:
import pandas as pd
import tabulate 
import pymysql

In [6]:
conn = pymysql.connect(host='localhost', user='root', password='b8SO0ibY5claaghlgiehgld', database='extremesportshouse')

cursor = conn.cursor()

In [7]:
cursor.execute("SELECT DATABASE();")

result = cursor.fetchone()
print("Currently using the database:", result[0])

Currently using the database: extremesportshouse


# **<font color = '#3498eb'>DATA LOADING AND UNDERSTANDING</font>**

# **<font color = '#3498eb'>DATA QUALITY</font>**